## Loss calculation example

In [244]:
import chainer as c

In [1]:
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions

## IOU computation

In [7]:
#%pdb

#from chainer.functions import maximum, minimum

def iou(boxA, boxB):
    
    
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min((boxA[0]+boxA[2]), (boxB[0]+boxB[2]))
	yB = min((boxA[1]+boxA[3]), (boxB[1]+boxB[3]))
 
	interArea = (xB - xA) * (yB - yA)
 
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = boxA[2]*boxA[3]
	boxBArea = boxB[2]*boxB[3]
 
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / (boxAArea + boxBArea - interArea)
 
	# return the intersection over union value
	return iou

## Naive matching in python and chainer

In [148]:
max_boxes = 4


boxes1 = Variable(np.array([[ 0.,  0.,  6.,  4., 1., 0.], [0., 0., 4., 4., 0., 1.]]))
boxes2 = Variable(np.array([[ 0.,  0.,  4.,  4., 0, 1], [1., 1., 1., 1., 1, 0]]))

# Construct matches: CPU
matches = np.zeros((boxes1.shape[0], boxes2.shape[0]))

positives = np.zeros((boxes1.shape[0], boxes2.shape[0]))
negatives = np.zeros((boxes1.shape[0], boxes2.shape[0]))

for i, box1 in enumerate(boxes1):
    for j, box2 in enumerate(boxes2):
        #print(iou(box1.data, box2.data))
        overlap = iou(box1.data, box2.data)
        matches[i][j] = overlap
        
        """ Check connectivity """
        if overlap>0.5:
            if np.argmax(box1.data[4:]) == np.argmax(box2.data[4:]):
                positives[i][j] = overlap
            else:
                negatives[i][j] = overlap
        
# Construct positive negative loss kernel : CPU
print("matches: ", matches)
print("positive matches: ", positives)
print("negative matches: ", negatives)

# Metrics on the positive array
#matches.

# Positive boxes indices to sample

# Negative boxes indices to sample

# Construct loss matrix by appending variable: CPU

# Construct loss variable as a for loop by appending positive negative

#print(iou(boxes1[0], boxes2[0]))
#boxA = boxes1[0]
#boxB = boxes2[0]
#min(boxA[0]+boxA[2], boxB[0]+boxB[2])

('matches: ', array([[ 0.66666667,  0.04166667],
       [ 1.        ,  0.0625    ]]))
('positive matches: ', array([[ 0.,  0.],
       [ 1.,  0.]]))
('negative matches: ', array([[ 0.66666667,  0.        ],
       [ 0.        ,  0.        ]]))


In [149]:
#matches.argsort()

""" Get sorted indices """
positive_indices = np.dstack(np.unravel_index(np.argsort(positives.ravel()), (2,2)))
negative_indices = np.dstack(np.unravel_index(np.argsort(negatives.ravel()), (2,2)))

""" Get counts of both positive and negative """
positive_count = np.count_nonzero(positives)
negative_count = np.count_nonzero(negatives)

print(positive_indices, negative_indices)
print(positive_count, negative_count)

(array([[[0, 0],
        [0, 1],
        [1, 1],
        [1, 0]]]), array([[[0, 1],
        [1, 0],
        [1, 1],
        [0, 0]]]))
(1, 1)


In [150]:
## Positive indices give us the positive indices
tosample = positive_count

positive_indices_sampled = positive_indices[0][0:tosample]
negative_indices_sampled = negative_indices[0][0:tosample]

## Loss calculation from indices

In [215]:

from chainer.functions import softmax_cross_entropy, cast, mean_squared_error

## Let's iterate over the positive indices and negative indices and add up the loss
loss_class = 0.
loss_reg = 0.

for i, j in negative_indices_sampled:
    p1 = boxes1[i][4:].reshape(1,2)
    p2 = argmax(boxes2[j][4:]).reshape(1)
    
    """ Softmax error """
    loss_class += softmax_cross_entropy(p1, p2)
    
    """ Mean squared error between the two boxes """
    box1 = cast(boxes1[0][:4], np.float32)
    box2 = cast(boxes2[0][:4], np.float32)
    loss_reg += mean_squared_error(box1, box2)
    #print(i,j)
    
#print(positive_indices_sampled, negative_indices_sampled)

#loss_class += np.dot(boxes1[i], boxes2[j])

print(loss_class, loss_reg)

(variable(0.3132616875182228), variable(1.0))


## YES! Loss calculation finally works, let's verify the calculations

## Now we have loss working for box variable. Let's add in the rest of the network

In [205]:
boxes1[i][4:].reshape(1,2).shape[2:]

()

## Generating randomized box predictions and hooking it up to our loss function

In [153]:
xx

variable([], shape=(0, 6))

In [159]:
x = np.linspace(-10, 10, 100).astype(np.float32)
y = 2 * x + 1 + np.random.randn(len(x)).astype(x.dtype) * 5
x = x.reshape(len(x), 1)
y = y.reshape(len(y), 1)

In [162]:
softmax_cross_entropy(x,y)

InvalidType: 
Invalid operation is performed in: SoftmaxCrossEntropy (Forward)

Expect: in_types[1].dtype == <type 'numpy.int32'>
Actual: float32 != <type 'numpy.int32'>

In [165]:
from chainer import datasets
iris = datasets.load_iris()
x = iris.data.astype(np.float32)
y = iris.target.astype(np.int32)

AttributeError: 'module' object has no attribute 'load_iris'

In [177]:
boxes2[1][4:].reshape(1,2)

variable([ 1.,  0.])

variable(1.0)